# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [366]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras import models
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.utils.vis_utils import model_to_dot
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import fbeta_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import cross_val_score
from IPython.display import SVG
import livelossplot
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

ModuleNotFoundError: No module named 'livelossplot'

### 1. Data Exploration (Portfolio Data)

In [ ]:
#Printing top5 rows from portfolio data
portfolio.head()

In [ ]:
#Printing number of rows and columns in portfolio dataframe
print("portfolio data: Rows = {0}, Columns = {1}".format(str(portfolio.shape[0]), str(portfolio.shape[1])))

In [ ]:
#Showing details in form of quantile, max, min , mean, standard deviation etc
portfolio.describe()

In [ ]:
#Printing datatype info about attributes of portfolio dataset
portfolio.info()

In [ ]:
# Checking the null values
portfolio.isnull().sum()

In [ ]:
# Exploring different attributes under channel column in portfolio dataset
portfolio.channels

#### 1.1 Portfolio Data Cleaning

In [ ]:
# First of all we need to separate the values of channels attribute and create separate table for values of channels 
# I will apply the concept of dummy columns, I will rename the column "id" as "offer_id", as its part of portfolio's offer.
"""
.apply(pd.Series) converts the series of lists to a dataframe
.stack() puts everything in one column again (creating a multi-level index)
pd.get_dummies( ) creating the dummies
.sum(level=0) for remerging the different rows that should be one row (by summing up the second level, only keeping the original level (level=0))
"""
df1= portfolio.copy()

dummy = pd.get_dummies(df1.channels.apply(pd.Series).stack()).sum(level=0)
df1 = pd.concat([df1, dummy], axis=1)
df1 = df1.drop(columns='channels')
df1 = df1.rename(columns={'id':'offer_id'})
df1.head()


### 2.Data Exploration (Profile Data)

In [ ]:
# Printing top5 rows of profile data
profile.head()

In [ ]:
#Row and Column detail about profile data
profile.shape

In [ ]:
#Quantile, min, max etc info about profile data
profile.describe()

In [ ]:
profile.describe(include="all")

In [ ]:
# Printing all values under age attribute of profile dataset
profile['age'].unique()

In [ ]:
# Datatype information of profile attributes
profile.info()

In [ ]:
# Checking the null values
profile.isnull().sum()

#### 2.1 Profile data cleaning

In [ ]:
# Checking the type of became_member_on attribute, as it contains value in the format of yy/mm/dd
type(profile.became_member_on[0])

In [ ]:
# Creating copy of dataframe for further implementaion, we need to change the type of "became_member_on" attribute 
# We will change the column name "id" to "customer_id"
df2 = profile.copy()

df2['became_member_on'] = pd.to_datetime(df2['became_member_on'], format='%Y%m%d')

df2 = df2.rename(columns={'id':'customer_id'})
df2.head(15)


#####  From above it looks like, that all null values belong to "age=118"

In [ ]:
# Checking the above fact
df2[df2['age']== 118].drop(['became_member_on', 'customer_id'], axis=1)

In [ ]:
# Cross checking the type of "became_member_on" attribute
type(df2.became_member_on[0])

### 3. Data Exploration (Transcript Data)

In [ ]:
# Print the transcript data 
transcript.head()

In [ ]:
# Printing rows and columns of transcript.json file
transcript.shape

In [ ]:
# Printing quantile, mean, max,min etc about transcript file
transcript.describe(include="all")

In [ ]:
# Displaying information about data type of transcript attributes
transcript.info()

In [ ]:
# Displaying the type of value and number of values against offer id
transcript['value'].value_counts()

In [ ]:
# Displaying the type of event and number of those event
transcript['event'].value_counts()

In [ ]:
# Checking the null value. if it is present in transcript data set
transcript.isnull().sum()

#### 3.1 Cleaning Transcript Data

In [ ]:
"""
First we will create another copy of transcript dataset like profile and portfolio.
Will chnage the column name 'person' to 'customer_id'
Convert the column 'event' into 4 different columns, based on their types. Will apply the concept of dummy.

"""
df3 = transcript.copy()

df3['event'] = df3['event'].str.replace(' ', '-')

df3 = df3.rename(columns={'person':'customer_id'})
dummy = pd.get_dummies(df3['event'])
df3 = pd.concat([df3, dummy], axis=1 )
df3.head()


In [ ]:
# This code is picked from https://stackoverflow.com/questions/60106364/syntaxerror-invalid-syntax-when-using-lambda-function-in-pandas-apply
df3['offer_id'] = [[*i.values()][0]if [*i.keys()][0] in ['offer id','offer_id'] else None for i in df3.value]
df3['amount'] = [np.round([*i.values()][0], decimals=2)if [*i.keys()][0] == 'amount' else None for i in df3.value]

In [ ]:
# Dropping the value column 
df3 = df3.drop(columns='value')
df3.head()

### Now We are done with data exploration and cleaning for all three datasets. Now as per need we will combine the data from all three datasets and draw visualization.

### Based on common attributes of all three datasets, we will try to merge the datasets.

In [ ]:
# First will start to merge transcript and profile data on 'cutomer id'
df_merge= pd.merge(df3, df2, on='customer_id')
df_merge.head()

In [ ]:
# Merge the portfolio dataset with df_merge dataframe
df = pd.merge(df_merge, df1, on='offer_id', how='left')
df

In [ ]:
df.columns

In [ ]:
# Lets try some graph based on the findings of so far, before that lets find mean value
df['income'].mean()

In [ ]:
type(df['time'])

In [ ]:
# distplot plot based on time attribute 
# This code is picked from https://indianaiproduction.com/seaborn-histogram-using-seaborn-distplot/
"""
Time attribute is related to offer_id, customer_id and other important attributes directly, so its 
important here to check maximum number of offers provided for particular category, as of now I am not 
defining 'time' attribute with respect to other attribute. I am just considering 'time' attribute alone.
"""
plt.figure(figsize=(16,9))
sns.set()
 
# hist keyword argument to change hist format
sns.distplot(df["time"],
            hist_kws = {'color':'#DC143C', 'edgecolor':'#aaff00',
                       'linewidth':5, 'linestyle':'--', 'alpha':0.9})

In [ ]:
df['offer_id'].value_counts().plot.barh(title=' Distribution of offer_ids')


In [ ]:
#Number of rows and columns in df dataframe
df.shape

In [ ]:
# Detailed information about df dataframe in terms of quantile, mean, deviation, min, max etc
df.describe(include='all')

In [ ]:
#Data type information about the attributes of dataframe
df.info()

In [ ]:
df['offer_id'].nunique()

In [ ]:
# Find all unique values of offer_id
offer_id = df['offer_id'].unique()
offer_id

In [ ]:
# Convert the offer_id values into dictionary form
offer_dict = pd.Series(offer_id ).to_dict()
offer_dict

In [ ]:
offer_dict = dict([(value, key) for key, value in offer_dict.items()]) 
offer_dict

In [ ]:
# mapping the offer_id with offer_dict, in order to get  
df['offer_id'] = df['offer_id'].map(offer_dict)
df

In [ ]:
df['offer_id'].unique()

In [ ]:
event_id = df['event'].unique()
event_id

In [ ]:
#in order to make compare and visualize all factors related to company promotions, like offers, event, we need to create co relation 
# between all factors, so that we can create more clear visualization. Here I am creating separate column, called event_id
# which is mapped according to event attribute.
event_dict = pd.Series(event_id).to_dict()

event_dict = dict([(value, key) for key, value in event_dict.items()]) 
df['event_id'] = df['event'].map(event_dict)
df

In [ ]:
# Lets cross check structure of dataframe
df.shape

In [ ]:
#converting the dataframe into csv data
df.to_csv('data/data.csv', index=False)


In [ ]:
data = pd.read_csv('data/data.csv')
data

In [ ]:
# Exploring data file
data.columns

In [ ]:
data.gender.unique()

In [ ]:
data.age.describe()

## 4. Visualization of cleaned data

In [ ]:
def horizontal_bar_plot(data, colName, pltTitle, pltfigsize=[5,5], bInvertYAxis=True, fntSize=12, fntWeight='bold'):
    """
    Creates a horizontal bar plot with counts of categories
    
    Parameters
    ----------
    df: The dataframe with categories.
    colName: The name of the column containing the categories.
    pltTitle: The plot title.
    pltfigsize: The figure size of the plot (default is [5,5]).
    bInvertYAxis: A flag to indicating whether to invert the Y-Axis or not.
    fntSize: The font size of text (default to 12 point font size)
    fntWeight: The weight of font (default to 'bold', it could be 'italic')
    
    Returns
    -------
    None
    """
    plt.figure(figsize=pltfigsize)
    fig, ax = plt.subplots()    
    y_counts = data[colName].value_counts()
    if bInvertYAxis == True:
        y_counts.plot(kind='barh').invert_yaxis()
    else:
         y_counts.plot(kind='barh')   
    for i, v in enumerate(y_counts):
        ax.text(v, i, str(v), color='black', fontsize=fntSize, fontweight=fntWeight)
        plt.title(pltTitle)

### Visualization #1

In [ ]:
# Pulling number of event categories as per cleaned data
horizontal_bar_plot(data, 'event', "Break-down of available event in data")

#### Observation : From above graph it is clear that only approx half offers reach to audience and out of that only 1/3 offers are completed by customers. Many offers dont even get seen after receiving.  

### Visualization#2

In [ ]:
horizontal_bar_plot(data, 'offer_type', "Break-down of available offer types in data")

#### Observation: Bogo and discount offers have similar distributions, but Bogo comes on top. Thats why many patterns are coming based on Bogo in market.

### Visualization#3

In [ ]:
# Using histogram trying to pull outliers
data.age.hist(bins = 20, grid=True)
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.title('Age Group Distribution');

In [ ]:
# Using box plot trying to pull outliers
sns.boxplot(profile['age'], width=0.5);

### Observation : 
##### Based on above two plots (histogram & Boxplot), we can see that age>100 is present , which is not possible. So will consider values above 
#### 100 as outlier. Age group between 47-62 use application the most. 63-71 age group uses the Starbuck app second most, as any
#### one may think here that younger age group must be using the app mostly. But this is not true here.

### Visualization#4

In [ ]:
# Using histogram trying to pull outliers
data.income.hist(bins = 20, grid=True)
plt.xlabel('Income Range')
plt.ylabel('Count')
plt.title('Income Range Distribution');

#### Observation : Average income is 65000-74000

### Visualization#5

In [ ]:
data['gender'].value_counts()

In [ ]:
data['gender'].value_counts().sum()

In [ ]:
print("Male gender population  : {} % " .format(str(data.gender.value_counts()[0] / data.shape[0]*100)))
print("Female gender population  : {} % " .format(str(data.gender.value_counts()[1] / data.shape[0]*100)))
print("Other gender population  : {} %" .format(str(data.gender.value_counts()[2] / data.shape[0]*100)))

In [ ]:
horizontal_bar_plot(data, 'gender', "Break-down of gender types in data")

#### Observation : Male is playing major role towards using Starbucks app

### Visualization#6

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(x= "offer_type", hue= "gender", data=data)
sns.set(style="darkgrid")
plt.title('Gender distribution as per offer_type')
plt.ylabel('Count')
plt.xlabel('Offer_Type')
plt.xticks(rotation=0)
plt.legend(title='Gender')

#### Observation : As we can see here , that Male are using the all types of offers mostly. Informational type offer is close for male and female.

### Visualization# 7

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(x= "event", hue= "gender", data=data)
sns.set(style="darkgrid")
plt.title('Gender distribution as per Event type')
plt.ylabel('Count')
plt.xlabel('Event')
plt.xticks(rotation=0)
plt.legend(title='Gender')

#### Observation: Here are few intresting facts about event type disrtibution among the gender. As we can see that there is huge gap between offer-received and offer-completed. But under offer-completed event, male and female got almost same number of count. Which indicates that male usually ignores the offers mostly or dont pay attention towards it, whereas female pays more attention towards offer. Which seems to be correct in real life as well :)

### Visualization# 8

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(x= "event", hue= "offer_type", data=data)
sns.set(style="whitegrid")
plt.title('Event type distribution as per offer_type')
plt.ylabel('Count')
plt.xlabel('Event')
plt.xticks(rotation=0)
plt.legend(title='Offer_type')

#### Observation : As we can see in above chart, that bogo and discount offers get receive in almost same quantity, but discount offers have less views compared to bogo offer, still discount offers are larger then bogo offer in terms of offer-completed category. 

### Visualization# 9

In [ ]:
offer_received = data[data['offer-received'] == 1].offer_id.value_counts()
offer_viewed = data[data['offer-viewed'] == 1].offer_id.value_counts()
offer_completed = data[data['offer-completed'] == 1].offer_id.value_counts()

 
print("Offer-Received as per types : {}".format(str(offer_received)))
print("Offer-Viewed as per types : {}".format(str(offer_viewed)))
print("Offer-Completed as per types : {}".format(str(offer_completed)))

In [ ]:
# Defining three subplots with grid size 1*3 for 1st subplot, with grid size 1*3 for 2nd subplot, with 1*3 grid size for 3rd subplot.
plt.subplot(1,3,1)
offer_received = data[data['offer-received'] == 1].offer_id.value_counts()
offer_received.plot(kind='barh', figsize=(15,5))
plt.ylabel('Offer Id')
plt.xlabel('count')
plt.title('Offer received with Offer Id ');

plt.subplot(1,3,2)
offer_viewed = data[data['offer-viewed'] == 1].offer_id.value_counts()
offer_viewed.plot(kind='barh' , figsize=(15,5))
plt.ylabel('Offer Id')
plt.xlabel('count')
plt.title('Offer viewed with Offer Id ');

plt.subplot(1,3,3)
offer_completed = data[data['offer-completed'] == 1].offer_id.value_counts()
offer_completed.plot(kind='barh' , figsize=(15,5))
plt.ylabel('Offer Id')
plt.xlabel('count')
plt.title('Offer completed got with Offer Id ');

#### Observation: From above subplots we can analyze that all Offer id received equal offers and 60% offer ids did view the offer in terms of good count, rest 40% is on average mark. If we talk about offer completion with respect to offer ids, then its showing considerably good count for all offer ids. 

### Visualization# 10

In [ ]:
# Here I am pulling three platforms where offers get shoot up mostly, email, mobile and social platforms.
email_received = df1[df1['email'] == 1].offer_id.value_counts()
mobile_received = df1[df1['mobile'] == 1].offer_id.value_counts()
social_received = df1[df1['social'] == 1].offer_id.value_counts()

 
print("email-Received as per types : {}".format(str(email_received)))
print("mobile-Viewed as per types : {}".format(str(mobile_received)))
print("social-Completed as per types : {}".format(str(social_received)))

In [ ]:
# Defining three subplots with grid size 1*3 for 1st subplot, with grid size 1*3 for 2nd subplot, with 1*3 grid size for 3rd subplot.
plt.subplot(1,3,1)
email = df1[df1['email'] == 1].offer_id.value_counts()
email.plot(kind='bar')
plt.ylabel('email')
plt.xlabel('Offer Id')
plt.title('Offer id linked with email ');

plt.subplot(1,3,2)
mobile = df1[df1['mobile'] == 1].offer_id.value_counts()
mobile.plot(kind='bar' , figsize=(15,5))
plt.ylabel('mobile')
plt.xlabel('Offer Id')
plt.title('Offer id linked with mobile ');

plt.subplot(1,3,3)
social = df1[df1['social'] == 1].offer_id.value_counts()
social.plot(kind='bar' , figsize=(15,5))
plt.ylabel('social')
plt.xlabel('Offer Id')
plt.title('Offer id linked with social ');

#### Observation : As we can see here offers get shoot up on email mostly. or can say first preference for Starbucks. Second its Mobile, and its also very popular medium for shooting offers. Mobile platform is almost equal to email. Social platform is bit low in terms of shooting offers, as its showing 60% compared to email. So may be Starbucks can consider to boost up the offers over social platform.

## 5. Build a Machine Learning model to predict response of a customer to the offer

##### Applying one hot encoding for gender and offer_type column, its the part of procedure to prepare the Pre Modal

In [ ]:
# Classifying gender categories and mapping to existing gender categories
genders = {'O': 0, 'M': 1, 'F': 2}
data['gender'] = data['gender'].map(genders)

In [ ]:
data.offer_type.value_counts()

In [ ]:
## Classifying offer_type categories and mapping to existing offer_type categories
offers = {'bogo': 0, 'discount': 1, 'informational': 2}
data['offer_type'] = data['offer_type'].map(offers)

In [ ]:
data.head()

In [ ]:
# Inorder to create test and train variables for testing and training , we need to look into "data" dataframe again and arrange
# the columns as per need.
data.columns

In [ ]:
X = data.drop(['customer_id', 'event_id' , 'event' , 'became_member_on','offer-completed', 'offer-received','offer-viewed', 'transaction'], axis=1)
Y = data['event_id'] 

In [ ]:
X.head()

In [ ]:
X['income'].mean()

In [ ]:
X['income'].fillna(X['income'].mean(), inplace=True)

In [ ]:
X['income'].isnull().sum()

In [ ]:
X.head()

In [ ]:
Y.head()

In [ ]:
X.describe()

In [ ]:
Y.describe()

In [ ]:
X.info()

In [ ]:
X.shape , Y.shape

#### Feature Engineering 


In [ ]:
class_name =['offer recieved', 'offer viewed', 'transaction', 'offer completed']

In [ ]:
#split the dataset into test and train sets.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
std = StandardScaler()

In [ ]:
X_train.income = std.fit_transform(X_train.income.values.reshape(-1, 1))
X_train.age = std.fit_transform(X_train.age.values.reshape(-1, 1))

X_train.reset_index(inplace=True)
X_train = X_train.drop(['index'], axis=1)

In [ ]:
X_train.income = std.fit_transform(X_train.income.values.reshape(-1, 1))
X_train.age = std.fit_transform(X_train.age.values.reshape(-1, 1))

X_train.reset_index(inplace=True)
X_train = X_train.drop(['index'], axis=1)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
y_train[0:1].shape

#### Building the Modal

In [ ]:
ann = keras.models.Sequential()

In [ ]:
NUM_ROWS = 100
NUM_COLS = 100
ann.add(keras.layers.Dense(6, activation='relu', input_shape=(NUM_ROWS * NUM_COLS,)))
#ann.add(Dropout(0.5))
ann.add(keras.layers.Dense(6, activation='relu'))
#ann.add(Dropout(0.25))
ann.add(keras.layers.Dense(4, activation = 'softmax'))



In [ ]:
ann.compile(optimizer = 'adam', 
            loss = 'sparse_categorical_crossentropy', 
            metrics = ['accuracy'])

In [ ]:
ann_history = ann.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=100)
#plot_losses = livelossplot.PlotLossesKeras()

 

#ann.fit(X_train, y_train, epochs=15, batch_size=100, verbose=1)

#ann.fit(X_train, y_train, batch_size=128, epochs=15, verbose=1, callbacks=[], validation_split=0.0, validation_data=(X_test, y_test), shuffle=True, class_weight=None, sample_weight=None)
